In [2]:
import numpy as np
from sklearn.metrics import precision_score, roc_auc_score
import pickle

In [57]:
gt_1 = np.genfromtxt('/app/All_models/model_pred_and_gt/DNNTSP/instacart_preprocessed/run_1/gt_test/data.csv', delimiter=',')
gt_2 = np.genfromtxt('/app/All_models/model_pred_and_gt/LANET/instacart_preprocessed/run_1/gt_test/data.csv', delimiter=',')

# preds = np.genfromtxt('/app/All_models/model_pred_and_gt/TCMBN/synthea_preprocessed/run_1/pred_test/data.csv', delimiter=',')


In [58]:
gt_1.shape

(2000, 217)

In [59]:
gt_2.shape

(2000, 134)

In [12]:
import os
import json
import torch
import itertools

# from utils.load_config import config

import numpy as np
import pickle


# def get_attribute(attribute_name: str, default_value=None):
#     """
#     get configs
#     :param attribute_name: config key
#     :param default_value: None
#     :return:
#     """
#     try:
#         return getattr(config, attribute_name)
#     except KeyError:
#         return default_value


# config.data_path = config.data

# dataset specified settings
# config.update(config[f"{get_attribute('dataset_name')}"])
# config.pop('JingDong')
# config.pop('DC')
# config.pop('TaoBao')
# config.pop('TMS')
# config.pop('JingDong_inductive')
# config.pop('DC_inductive')
# config.pop('TaoBao_inductive')
# config.pop('TMS_inductive')
# config.pop('mimic3_preprocessed')

def unpickle_file(path, type_of_split, data, prefix, encoding):
    tmp_path = os.path.join(path, prefix + type_of_split + '.pkl')
    with open(tmp_path, 'rb') as file:
        data[type_of_split] = pickle.load(file, encoding=encoding)[type_of_split]
    return data

def retrieve_dict(path, prefix='', encoding='ASCII'):
    tmp_path = os.path.join(path, prefix + 'dev.pkl')
    # print(encoding)
    with open(tmp_path, 'rb') as file:
        data = pickle.load(file, encoding=encoding)
    unpickle_file(path, 'train', data, prefix, encoding=encoding)
    unpickle_file(path, 'test', data, prefix, encoding=encoding)
    return data

def from_one_hot(one_hot_encoded_basket: list) -> list:
    return np.nonzero(one_hot_encoded_basket)[0].tolist()

def convert_dataset_from_pickle(unpickled_data) -> None:
    converted_data = {'train': [], 'validate': [], 'test': []}  # Assuming you might have 'validate' and 'test' sets as well
    
    # Convert each dataset section (train, validate, test)
    help_dict = {'train' : 'train', 'dev' : 'validate', 'test' : 'test'}
    for section in help_dict:
        for user_id, baskets in enumerate(unpickled_data[section]):
            user_baskets = []

            # if isinstance(user_id, str):
            #     if user_id.startswith('user'):
            #         user_id = int(user_id[4:])
            #     else:
            #         user_id = int(user_id)
                

            set_time = 1  # Initialize a time counter; you might have real timestamps to use instead
            for basket in baskets:
                if ~(np.all(basket['type_event'] == 0)):
                    user_baskets.append({
                        "user_id": user_id,  # Convert user ID to an integer
                        "items_id": from_one_hot(basket['type_event']),       # The list of item IDs in the basket
                        "set_time": basket['time_since_start']      # The time the basket was 'purchased'
                    })

            # Add the converted user baskets to the respective section of the dataset
            converted_data[help_dict[section]].append(user_baskets)

    return converted_data

def get_users_items_num_and_max_seq_length(data_path):
            

    data = retrieve_dict(data_path)
    data_dict = convert_dataset_from_pickle(data)

    max_seq_length = -1
    # get users and items num
    user_ids_set, item_ids_set = set(), set()
    for data_type in data_dict:
        for user_sets in data_dict[data_type]:
            user_ids_set = user_ids_set.union({user_sets[0]['user_id']})
            item_ids_set = item_ids_set.union(set(itertools.chain.from_iterable([user_set['items_id'] for user_set in user_sets])))

            if len(user_sets) - 1 > max_seq_length:
                max_seq_length = len(user_sets) - 1

    num_users, num_items = len(user_ids_set), len(item_ids_set)

    return num_users, num_items, max_seq_length


num_users, num_items, max_seq_length = get_users_items_num_and_max_seq_length('/app/All_models/tcmbn_data/DC_preprocessed/split_1')
# config.device = f'cuda:{get_attribute("cuda")}' if torch.cuda.is_available() and get_attribute("cuda") >= 0 else 'cpu'


In [14]:
num_items

217

In [13]:
gt_1.shape

(2000, 134)

In [10]:
gt_2.shape

(9010, 217)

In [ ]:
with open('/app/All_models/tcmbn_data/synthea_preprocessed/split_1/test.pkl', 'rb') as f:
    data = pickle.load(f, encoding='ascii')

In [148]:
s = np.zeros(232)

for user in data['test']:
    for basket in user:
        s += basket['type_event']

In [149]:
s

array([1.370e+02, 2.880e+02, 2.510e+02, 8.000e+00, 1.040e+02, 2.400e+01,
       8.820e+02, 7.000e+00, 9.900e+01, 1.800e+01, 6.000e+00, 9.300e+01,
       5.300e+01, 4.500e+01, 2.700e+01, 5.000e+00, 1.000e+01, 4.000e+00,
       5.500e+01, 6.000e+00, 6.000e+00, 1.800e+01, 2.300e+01, 6.000e+00,
       6.000e+01, 1.600e+01, 7.000e+00, 3.530e+02, 6.000e+00, 7.000e+00,
       1.700e+01, 6.000e+00, 2.500e+02, 1.200e+01, 4.400e+01, 1.200e+01,
       3.500e+02, 7.000e+00, 5.000e+00, 1.100e+01, 1.680e+02, 1.300e+01,
       4.000e+00, 2.020e+02, 1.100e+01, 1.100e+01, 1.248e+03, 2.780e+02,
       2.000e+00, 5.580e+02, 7.000e+00, 1.510e+02, 4.300e+01, 5.000e+00,
       7.000e+00, 2.800e+01, 5.600e+01, 3.290e+02, 1.000e+00, 1.300e+01,
       1.919e+03, 2.440e+02, 6.000e+00, 9.900e+01, 1.600e+01, 5.300e+01,
       3.850e+02, 6.000e+00, 1.199e+03, 4.140e+02, 2.600e+01, 8.120e+02,
       6.600e+01, 6.420e+02, 1.000e+01, 3.000e+00, 5.450e+02, 3.170e+02,
       1.600e+01, 5.110e+02, 1.027e+03, 6.100e+01, 

In [143]:
gt.shape

(1327, 169)

In [144]:
gt.sum(axis=0)

array([  4., 234., 125.,   7.,  82.,   7.,   6.,  14.,   7.,   6.,  26.,
         6.,   5.,   6.,  29.,  13.,  15.,   4.,   8.,  32.,   5.,  22.,
         0.,   9.,   6.,  27.,  22.,  11.,   2.,  23.,  14.,   3.,  11.,
         5., 290., 160.,  16.,   2., 123.,   6., 115., 129.,  24.,  60.,
        10.,  15.,  19., 131.,  19.,  21.,  37.,   8.,  10.,  12.,   1.,
         3., 129.,  12., 265.,  89., 448.,  11.,  28.,   6.,  92., 436.,
        15., 298.,  28.,  29.,  58.,  36.,  19.,  11.,  55., 119.,  18.,
        41.,  40., 122.,   6., 330.,  38.,  38.,  56.,   6., 235.,  27.,
        29.,  19.,  13.,  17.,  49.,   6.,  32.,  30.,  38.,  33., 128.,
        26.,  37.,   7.,   5.,  15.,  14.,  61.,  52.,   3.,   0.,   2.,
         2., 121.,  28.,  43.,  24.,  23.,  61.,  16.,   2.,  13.,  99.,
        25.,   3.,   7.,   7.,   2.,   8.,   4.,   4.,   3.,  13.,  30.,
         5.,   4., 194., 260.,   4.,   1.,  76.,  21.,  12., 162.,  17.,
        14.,  14.,   2., 265., 322.,  69.,   3.,  3

In [96]:
import json

In [150]:
with open('/app/All_models/dnntsp_data/synthea/synthea.json', 'r') as f:
    dnntsp_dict = json.load(f)

In [151]:
a = dnntsp_dict['test'].values()

In [152]:
occ_dict = {i: 0 for i in range(0,232)}
for user in dnntsp_dict['test'].values():
    # print(user)
    for basket in user:
        # print(basket)
        for item in basket:
            occ_dict[item] += 1

In [153]:
occ_dict

{0: 137,
 1: 288,
 2: 251,
 3: 8,
 4: 104,
 5: 24,
 6: 882,
 7: 7,
 8: 99,
 9: 18,
 10: 6,
 11: 93,
 12: 53,
 13: 45,
 14: 27,
 15: 5,
 16: 10,
 17: 4,
 18: 55,
 19: 6,
 20: 6,
 21: 18,
 22: 23,
 23: 6,
 24: 60,
 25: 16,
 26: 7,
 27: 353,
 28: 6,
 29: 7,
 30: 17,
 31: 6,
 32: 250,
 33: 12,
 34: 44,
 35: 12,
 36: 350,
 37: 7,
 38: 5,
 39: 11,
 40: 168,
 41: 13,
 42: 4,
 43: 202,
 44: 11,
 45: 11,
 46: 1248,
 47: 278,
 48: 2,
 49: 558,
 50: 7,
 51: 151,
 52: 43,
 53: 5,
 54: 7,
 55: 28,
 56: 56,
 57: 329,
 58: 1,
 59: 13,
 60: 1919,
 61: 244,
 62: 6,
 63: 99,
 64: 16,
 65: 53,
 66: 385,
 67: 6,
 68: 1199,
 69: 414,
 70: 26,
 71: 812,
 72: 66,
 73: 642,
 74: 10,
 75: 3,
 76: 545,
 77: 317,
 78: 16,
 79: 511,
 80: 1027,
 81: 61,
 82: 39,
 83: 73,
 84: 71,
 85: 162,
 86: 295,
 87: 75,
 88: 12,
 89: 66,
 90: 52,
 91: 68,
 92: 64,
 93: 8,
 94: 309,
 95: 135,
 96: 16,
 97: 210,
 98: 168,
 99: 189,
 100: 41,
 101: 708,
 102: 65,
 103: 26,
 104: 1248,
 105: 463,
 106: 168,
 107: 19,
 108: 30,
 1

In [11]:
non_trivial_targets = np.where((gt.sum(axis=0) != 0))[0]

In [14]:
len(non_trivial_targets)

130

In [15]:
roc_auc_score(gt[:, non_trivial_targets], preds[:, non_trivial_targets], average='weighted')

0.4874733134671268

In [6]:
test.sum(axis=0)

array([  4., 234., 125.,   7.,  82.,   7.,   6.,  14.,   7.,   6.,  26.,
         6.,   5.,   6.,  29.,  13.,  15.,   4.,   8.,  32.,   5.,  22.,
         0.,   9.,   6.,  27.,  22.,  11.,   2.,  23.,  14.,   3.,  11.,
         5., 290., 160.,  16.,   2., 123.,   6., 115., 130.,  24.,  60.,
        10.,  15.,  19., 131.,  19.,  21.,  37.,   8.,  10.,  12.,   1.,
         3., 129.,  12., 265.,  89., 448.,  11.,  28.,   6.,  92., 436.,
        15., 298.,  28.,  29.,  58.,  36.,  19.,  11.,  55., 119.,  18.,
        41.,  40., 122.,   6., 329.,  38.,  38.,  56.,   6., 235.,  27.,
        29.,  19.,  13.,  17.,  49.,   6.,  32.,  30.,  38.,  33., 128.,
        26.,  37.,   7.,   5.,  15.,  14.,  61.,  52.,   3.,   0.,   2.,
         2., 121.,  28.,  43.,  24.,  23.,  61.,  16.,   2.,  13.,  99.,
        25.,   3.,   7.,   7.,   2.,   8.,   4.,   4.,   3.,  13.,  30.,
         5.,   4., 193., 259.,   4.,   1.,  76.,  21.,  12., 162.,  17.,
        14.,  14.,   2., 265., 322.,  69.,   3.,  3

In [3]:
arr = test.reshape(-1, 24)

In [31]:
test

array([[0.10456368, 0.1045623 , 0.10456277, ..., 0.10456172, 0.10456245,
        0.10455842],
       [0.10456315, 0.10456277, 0.10455993, ..., 0.10456315, 0.10456212,
        0.10455993],
       [0.10456311, 0.10456288, 0.10455928, ..., 0.10456268, 0.10456235,
        0.10455928],
       ...,
       [0.10456324, 0.10456301, 0.1045602 , ..., 0.10456346, 0.10456236,
        0.1045602 ],
       [0.10456306, 0.10456304, 0.10455975, ..., 0.10456225, 0.10456206,
        0.1045598 ],
       [0.10456324, 0.10456296, 0.1045598 , ..., 0.10456296, 0.10456251,
        0.1045598 ]])

In [14]:
arr.sum(axis=0)

array([ 20.,   1.,  11.,   1.,  89., 242.,  10.,   2.,   0., 414.,  41.,
       111., 130.,  14.,   7.,  31.,  67., 183.,   0.,  16.,  11.,  33.,
         4.,   3.])

In [12]:
test_sfcn = np.genfromtxt('/app/SFCNTSP-onemore/model_pred_and_gt/DunnHumby/run_0/gt_test/data.csv', delimiter=',')

In [15]:
test_sfcn.sum(axis=0)

array([ 20.,   1.,  11.,   1.,  89., 242.,  10.,   2.,   0., 414.,  41.,
       111., 130.,  14.,   7.,  31.,  67., 183.,   0.,  16.,  11.,  33.,
         4.,   3.])

In [48]:
a = np.array([[1,0,1], [0,0,1]])
b = np.array([[0,0], [1,0]])

In [19]:
precision_score(a[:, 1], b[:, 1])

/app/miniconda3/envs/main/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [45]:
def get_non_trivial_targets(gt_test) -> np.ndarray:
    return np.where((gt_test.sum(axis=0) != 0))[0]

def get_masked(mask, *args):
    """in args expect arrays to get masked"""
    tmp = []
    for arg in args:
        tmp.append(arg[:, mask])
    return tuple(tmp)

In [49]:
mask = get_non_trivial_targets(a)

In [50]:
get_masked(mask, a)

(array([[1, 1],
        [0, 1]]),)

In [42]:
a.sum(axis=0) != 0

array([ True, False])

In [41]:
a

array([[1, 0],
       [0, 0]])

In [24]:
b[:, 1]

array([0, 0])

In [25]:
a[:, 1]

array([0, 0])

In [26]:
a

array([[1, 0],
       [0, 0]])

In [27]:
b

array([[0, 0],
       [1, 0]])